1) 뉴스 기사 제목 데이터에 대한 이해


In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [2]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [3]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


2) 텍스트 전처리

In [4]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df['clean_doc'][1]


"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

In [5]:
import nltk
nltk.download('stopwords')

# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

print(tokenized_doc[:5])

[nltk_data] Downloading package stopwords to /Users/yomce/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object


3) 정수 인코딩과 단어 집합 만들기

In [6]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0


[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


In [7]:
print(dictionary[66])
len(dictionary)

bye-bye,


181856

4. lda 모델 훈련시키기

In [8]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.007*"people" + 0.006*"government" + 0.005*"armenian" + 0.004*"state"')
(1, '0.016*"anyone" + 0.015*"please" + 0.013*"know" + 0.012*"windows"')
(2, '0.008*"people" + 0.008*"would" + 0.006*"believe" + 0.006*"jesus"')
(3, '0.004*"ottawa" + 0.003*"boston," + 0.003*"lemieux" + 0.003*"dallas"')
(4, '0.006*"amendment" + 0.005*"engine" + 0.004*"simms" + 0.003*"------"')
(5, '0.008*"medical" + 0.007*"health" + 0.006*"public" + 0.005*"keys"')
(6, '0.005*"sabbath" + 0.005*"u.s.a." + 0.003*"creed" + 0.002*"firearms,"')
(7, '0.008*"----------------------------------------------------------------------------" + 0.007*"banks" + 0.007*"gordon" + 0.006*"scsi-2"')
(8, '0.004*"stealth" + 0.003*"accelerator" + 0.003*"supports:" + 0.003*"creates"')
(9, '0.006*"excellent" + 0.006*"price:" + 0.005*"water" + 0.005*"smokeless"')
(10, '0.016*"would" + 0.010*"like" + 0.009*"think" + 0.007*"know"')
(11, '0.005*"picture" + 0.004*"present." + 0.002*"prints" + 0.002*"keller"')
(12, '0.015*"55.0" + 0.007*"unit

5. LDA 시각화 하기

In [10]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)


6. 문서별 토픽분포 보기

In [13]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)


0 번째 문서의 topic 비율은 [(0, 0.65818536), (2, 0.15273778), (10, 0.16098952), (18, 0.015928352)]
1 번째 문서의 topic 비율은 [(2, 0.53799987), (10, 0.4405137)]
2 번째 문서의 topic 비율은 [(0, 0.1743722), (2, 0.42004773), (10, 0.39115292)]
3 번째 문서의 topic 비율은 [(0, 0.31721613), (5, 0.062087588), (10, 0.43279344), (15, 0.105739035), (16, 0.04251715), (17, 0.029260665)]
4 번째 문서의 topic 비율은 [(2, 0.15151), (9, 0.03818065), (10, 0.3603828), (11, 0.24620132), (17, 0.17587727)]


In [16]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table._append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0,0.6582,"[(0, 0.6581754), (2, 0.15281807), (10, 0.16091..."
1,1,2,0.5380,"[(2, 0.53798896), (10, 0.4405246)]"
2,2,2,0.4200,"[(0, 0.17437978), (2, 0.41999793), (10, 0.3911..."
3,3,10,0.4328,"[(0, 0.31722632), (5, 0.062087737), (10, 0.432..."
4,4,10,0.3603,"[(2, 0.15155044), (9, 0.038180638), (10, 0.360..."
5,5,10,0.8079,"[(2, 0.14900184), (10, 0.80787456)]"
6,6,10,0.4663,"[(1, 0.1378537), (4, 0.1007404), (8, 0.0728001..."
7,7,10,0.7860,"[(2, 0.1192856), (10, 0.78596246), (13, 0.0164..."
8,8,10,0.3876,"[(5, 0.1439917), (8, 0.028454212), (10, 0.3876..."
9,9,10,0.6680,"[(1, 0.09566873), (4, 0.15837279), (10, 0.6680..."
